In [1]:
from cerebralcortex.util.helper_methods import get_study_names
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructField, StructType, DoubleType,MapType, StringType,ArrayType, FloatType, TimestampType, IntegerType
from pyspark.sql.functions import minute, second, mean, window
from pyspark.sql import functions as F
import numpy as np
import pandas as pd
from cerebralcortex.core.datatypes import DataStream
from cerebralcortex.core.metadata_manager.stream.metadata import Metadata, DataDescriptor, \
ModuleMetadata
from typing import List
import numpy as np
from scipy import signal
import pandas as pd
import pickle
from cerebralcortex import Kernel
from pyspark.sql import functions as F
CC = Kernel("/home/jupyter/cc3_conf/", study_name='mperf')

20 seconds data saving

In [ ]:
total_user_data,daywise_user_data = pickle.load(open('../data/data_participants.p','rb'))
wrist = 'left'
sensor_name='motionsensehrvmperf'
stream_name = "org.md2k."+str(sensor_name)+"."+str(wrist)+".wrist.acl.filtered"
data = CC.get_stream(stream_name)
users = list(total_user_data.keys())
data = data.filter(F.col('user').isin(users))
data  = data.withColumn('day',F.date_format('localtime',"yyyyMMdd"))
days = list(list(zip(*list(daywise_user_data.keys())))[1])
data = data.filter(F.col('day').isin(days))

data = data.withColumn('time',F.col('timestamp').cast('double'))

data = data.withColumn('data',F.array('time',
                                      'red','infrared','green',
                                      'aclx','acly','aclz',
                                      'gyrox','gyroy','gyroz')).drop(*['time','red','infrared','green','aclx',
                                                       'acly','aclz','gyrox','gyroy','gyroz'])

win = F.window("timestamp", windowDuration='20 seconds', startTime='0 seconds')
groupbycols = ["user",'version'] + [win]
data_grouped = data._data.groupBy(groupbycols).agg(F.collect_list('data'),
                                                   F.collect_list('timestamp'),
                                                  F.collect_list('localtime')).withColumnRenamed('collect_list(data)',
                                                                                                 'data').withColumnRenamed('collect_list(timestamp)',
                                                                                                                            'timestamp').withColumnRenamed('collect_list(localtime)',
                                                                                                                            'localtime')

data_grouped = data_grouped.withColumn('localtime',F.col('localtime').getItem(0)).withColumn('timestamp',F.col('timestamp').getItem(0))

stream_metadata = Metadata()
stream_name = "org.md2k."+str(sensor_name)+"."+str(wrist)+".wrist.acl.filtered.data.grouped.20secs"
stream_metadata.set_name(stream_name).set_description("Data Parsed as minute length array") \
    .add_dataDescriptor(
    DataDescriptor().set_name("window").set_type("object").set_attribute("description", \
    "start and end times")) \
    .add_dataDescriptor( \
    DataDescriptor().set_name("data").set_type("array").set_attribute("description", \
    "input data")) \
    .add_dataDescriptor( \
    DataDescriptor().set_name("user").set_type("string").set_attribute("description", \
    "user id")) \
    .add_dataDescriptor( \
    DataDescriptor().set_name("version").set_type("integer").set_attribute("description", \
    "version id")) \
    .add_dataDescriptor( \
    DataDescriptor().set_name("timestamp").set_type("timestamp").set_attribute("description", \
    "timestamp")) \
    .add_dataDescriptor( \
    DataDescriptor().set_name("localtime").set_type("timestamp").set_attribute("description", \
    "localtime")) \
    .add_module( \
    ModuleMetadata().set_name("fourtytwo/mullah/cc3/motionsenseHRVquality.ipynb").set_attribute("url", "https://md2k.org").set_author(
        "Md Azim Ullah", "mullah@memphis.edu"))
data_grouped = DataStream(data=data_grouped,metadata=stream_metadata)
CC.save_stream(data_grouped,overwrite=True)

Sample Minutes per day maintaining the proportion of data available for each participant

In [ ]:
def get_metadata(sensor_name,wrist):
    stream_metadata = Metadata()
    stream_name = "org.md2k."+str(sensor_name)+"."+str(wrist)+".wrist.acl.filtered.data.grouped.20secs.sampled"
    stream_metadata.set_name(stream_name).set_description("Data Parsed as minute length array") \
        .add_dataDescriptor(
        DataDescriptor().set_name("day").set_type("string").set_attribute("description", \
        "day as string")) \
        .add_dataDescriptor( \
        DataDescriptor().set_name("data").set_type("array").set_attribute("description", \
        "input data")) \
        .add_dataDescriptor( \
        DataDescriptor().set_name("user").set_type("string").set_attribute("description", \
        "user id")) \
        .add_dataDescriptor( \
        DataDescriptor().set_name("version").set_type("integer").set_attribute("description", \
        "version id")) \
        .add_dataDescriptor( \
        DataDescriptor().set_name("timestamp").set_type("timestamp").set_attribute("description", \
        "timestamp")) \
        .add_dataDescriptor( \
        DataDescriptor().set_name("localtime").set_type("timestamp").set_attribute("description", \
        "localtime")) \
        .add_module( \
        ModuleMetadata().set_name("fourtytwo/mullah/cc3/motionsenseHRVquality.ipynb").set_attribute("url", "https://md2k.org").set_author(
            "Md Azim Ullah", "mullah@memphis.edu"))
    return stream_metadata


def random_selection(data,total_user_data,daywise_user_data,minutes_total,sensor_name,wrist):
    schema = data.schema
    @pandas_udf(schema, PandasUDFType.GROUPED_MAP)
    def select_randomly(df):
        if (df['user'].values[0],df['day'].values[0]) not in daywise_user_data:
            return pd.DataFrame([],columns=df.columns)
        if df.shape[0]<20:
            return pd.DataFrame([],columns=df.columns)
        total = total_user_data[df['user'].values[0]]
        day_total = daywise_user_data[df['user'].values[0],df['day'].values[0]]
        selected_minutes = int(np.ceil(minutes_total*day_total/total))
#         df['data'] = df['data'].apply(lambda b:np.array([np.array(a) for a in b]))
        df['shape'] = df['data'].apply(lambda b: len(b))
        df = df[df['shape']>=498]
        df.drop(columns=['shape'],inplace=True) 
        if selected_minutes<df.shape[0]:
            df = df.sample(selected_minutes)
        else:
            df = df.sample(selected_minutes,replace=True)
        return df
    data = data._data.groupBy(['user','day','version']).apply(select_randomly)
    stream_metadata = get_metadata(sensor_name,wrist)
    data = DataStream(data=data,metadata=stream_metadata)
    return data

In [ ]:
total_user_data,daywise_user_data = pickle.load(open('../data/data_participants.p','rb'))
wrist = 'left'
sensor_name='motionsensehrvmperf'
minutes_total  = 200*3
stream_name = "org.md2k."+str(sensor_name)+"."+str(wrist)+".wrist.acl.filtered.data.grouped.20secs"
data = CC.get_stream(stream_name).drop('window')
data  = data.withColumn('day',F.date_format('localtime',"yyyyMMdd"))
data_final = random_selection(data,total_user_data,daywise_user_data,minutes_total,sensor_name,wrist)
CC.save_stream(data_final,overwrite=True)

Saving Participants Data

In [2]:
total_user_data,daywise_user_data = pickle.load(open('../data/data_participants.p','rb'))
wrist = 'left'
sensor_name='motionsensehrvmperf'
stream_name = "org.md2k."+str(sensor_name)+"."+str(wrist)+".wrist.acl.filtered.data.grouped.20secs.sampled"
data = CC.get_stream(stream_name)

# data.count()

# data.select('user').distinct().count()

# 48690/79

import pickle
for user in list(total_user_data.keys()):
    data = CC.get_stream(stream_name,user_id=user).toPandas()
    data['data'] = data['data'].apply(lambda b:np.array([np.array(a) for a in b]))
    pickle.dump(data,open('../data/data/'+str(user)+'.p','wb'))


# for i,row in data.iterrows():
#     print(row['data'].shape)
# data.head()

In [3]:
import shutil
shutil.make_archive('../data/data/','zip','../data/data/')

'/home/jupyter/mullah/person-identification/data/data.zip'